### CG Symmetric GS solver subroutine

In [1]:
import numpy as np
import math
from scipy.sparse import diags
from itertools import combinations

In [15]:
def generate_subroutine_bulk():
    ''' make 27pt stencil - bulk 3d '''
    outstring = "//bulk\n"
    outstring += "else{\n" 
    outstring += "xv[ix+iy*nx+iz*ny*nx] = (rv[ix+iy*nx+iz*ny*nx]\n"
    for i in range(-1,2):
        for j in range(-1,2):
            for k in range(-1,2):
                if i==j and j==k and i==0:
                    continue
                else:
                    outstring+="+ xv[ix+{} + (iy+{})*nx + (iz+{})*nx*ny]\n".format(i,j,k)
    outstring += ")/26.;}\n"
    return outstring

In [3]:
def zero_index_27pt():
    ''' corners '''
    def make_tempstring(l1,u1,l2,u2,l3,u3):
        tempstring = "xv[ix+iy*nx+iz*ny*nx] = (rv[ix+iy*nx+iz*ny*nx] \n"
        for n0 in range(l1,u1):
            for n1 in range(l2,u2):
                for n2 in range(l3,u3):
                    if(n0 == 0 and n1 ==n0 and n2==n1):
                        continue
                    tempstring += "+xv[ix+iy*nx+iz*ny*nx]\n".replace("i"+i1, "({}+{})".format("i"+i1,n0))\
                    .replace("i"+i2, "({}+{})".format("i"+i2,n1)).replace("i"+i3, "({}+{})".format("i"+i3,n2))
        tempstring += ")/26.;}\n"
        return tempstring
    i1,i2,i3 = 'x', 'y', 'z'
    outstring = ""

    def set_value(val, indx):
        if val == 0:
            return "i{} == 0".format(indx,indx)
        elif val == 1:
            return "i{} == n{}-1".format(indx,indx,indx)

    for ia in range(2):
        for ib in range(2):
            for ic in range(2):
                if (ia == 0 and ib == 0 and ic ==0):
                    outstring += "if("
                else:
                    outstring += "else if("
                outstring += set_value(ia, 'x') + "&& "
                outstring += set_value(ib, 'y') + "&& "
                outstring += set_value(ic, 'z') + "){\n"
                outstring += make_tempstring(-ia,-ia+2,-ib,-ib+2,-ic,-ic+2)

    return outstring

In [4]:
def one_index_27pt():
    ''' edges '''
    outstring = "//edges \n"
    dims = 'xyz'
    for c in combinations(dims, 2):
        i1,i2,i3 = [i for i in dims if i not in c][0],c[0], c[1]
        def make_tempstring(l1,u1,l2,u2):
            tempstring = "i{}>0 && i{}<n{}-1)".format(i1,i1,i1,i1) + "{\n"
            tempstring += "xv[ix+iy*nx+iz*ny*nx] = (rv[ix+iy*nx+iz*ny*nx] \n"
            for n0 in range(-1,2):
                for n1 in range(l1,u1):
                    for n2 in range(l2,u2):
                        if(n0 == 0 and n1 ==n0 and n2==n1):
                            continue
                        tempstring += "+xv[ix+iy*nx+iz*ny*nx]\n".replace("i"+i1, "({}+{})".format("i"+i1,n0))\
                        .replace("i"+i2, "({}+{})".format("i"+i2,n1)).replace("i"+i3, "({}+{})".format("i"+i3,n2))
            tempstring += ")/26.;}\n"
            return tempstring
        outstring += "else if (i{} == 0 && ".format(i2)
        outstring += "i{} == 0 &&".format(i3)
        outstring += make_tempstring(0,2,0,2)
        outstring += "else if (i{} == 0 && ".format(i2)
        outstring += "i{} == n{}-1 &&".format(i3,i3)
        outstring += make_tempstring(0,2,-1,1)
        outstring += "else if (i{} == n{}-1 && ".format(i2,i2)
        outstring += "i{} == 0 && ".format(i3)
        outstring += make_tempstring(-1,1,0,2)
        outstring += "else if (i{} == n{}-1 && ".format(i2,i2)
        outstring += "i{} == n{}-1 &&".format(i3,i3)
        outstring += make_tempstring(-1,1,-1,1)
        
    return outstring

In [12]:
def two_index_27pt():
    ''' sides '''
    outstring = "//sides \n"
    dims = 'xyz'
    for c in combinations(dims, 2):
        i1,i2,i3 = [i for i in dims if i not in c][0],c[0], c[1]
        def make_tempstring(l2,u2):
            tempstring = "i{}>0 && i{}<n{}-1 && ".format(i2,i2,i2,i2)
            tempstring += "i{}>0 && i{}<n{}-1)".format(i3,i3,i3,i3) + "{\n"
            tempstring += "xv[ix+iy*nx+iz*ny*nx] = (rv[ix+iy*nx+iz*ny*nx] \n"
            for n0 in range(-1,2):
                for n1 in range(-1,2):
                    for n2 in range(l2,u2):
                        if(n0 == 0 and n1 ==n0 and n2==n1):
                            continue
                        tempstring += "+xv[ix+iy*nx+iz*ny*nx]\n".replace("i"+i2, "({}+{})".format("i"+i2,n0))\
                        .replace("i"+i3, "({}+{})".format("i"+i3,n1)).replace("i"+i1, "({}+{})".format("i"+i1,n2))
            tempstring += ")/26.;}\n"
            return tempstring
        outstring += "else if (i{} == 0 && ".format(i1)
        outstring += make_tempstring(0,2)
        outstring += "else if (i{} == n{}-1 && ".format(i1,i1)
        outstring += make_tempstring(-1,1)
        
    return outstring

In [6]:
def start_blurb_forward():
    outstring = '''int ComputeSYMGS_ref( const SparseMatrix & A, const Vector & r, Vector & x) {

  assert(x.localLength==A.localNumberOfColumns); // Make sure x contain space for halo values

#ifndef HPCG_NO_MPI
  ExchangeHalo(A,x);
#endif

  local_int_t ix = 0;
  local_int_t iy = 0;
  local_int_t iz = 0;
  local_int_t nex = 0;
  local_int_t ney = 0;
  local_int_t nez = 0;
  local_int_t nx = A.geom->nx;
  local_int_t ny = A.geom->ny;
  local_int_t nz = A.geom->nz;
  local_int_t nlocal = nx*ny*nz;
  int npx          = A.geom->npx;
  int npy          = A.geom->npy;
  int npz          = A.geom->npz;
  int ipx          = A.geom->ipx;
  int ipy          = A.geom->ipy;
  int ipz          = A.geom->ipz;

//  const local_int_t nrow = A.localNumberOfRows;
//  double ** matrixDiagonal = A.matrixDiagonal;  // An array of pointers to the diagonal entries A.matrixValues
  const double * const rv = r.values;
  double * const xv = x.values;

// forward sweep
for( iz=0; iz< nz; iz++){
    for( iy=0; iy< ny; iy++){
        for (ix=0; ix < nx; ix++){
'''
    return outstring

In [20]:
def start_blurb_backwards():
    return '''// backwards sweep
for( iz=nz-1; iz>=0; iz--){
    for( iy=ny-1; iy>=0; iy--){
        for (ix=nx-1; ix>=0; ix--){
'''

In [8]:
def end_blurb():
    return '''      
        }
      }
    }
'''
    return outstring

In [18]:
def generate_subtroutine():
    return start_blurb_forward() + "\n" \
         + zero_index_27pt() + "\n" \
         + one_index_27pt() + "\n" \
         + two_index_27pt() + "\n" \
         + generate_subroutine_bulk() + "\n" \
         + end_blurb() +"\n"\
         + start_blurb_backwards() + "\n" \
         + zero_index_27pt() + "\n" \
         + one_index_27pt() + "\n" \
         + two_index_27pt() + "\n" \
         + generate_subroutine_bulk() + "\n" \
         + end_blurb() +"\n"\
         + '''  
return 0;
}'''

In [19]:
with open('/home/dcase/NCAS/SolverBenchmark/HPCG/breakdown_matrix_free_test/src/symgs_subroutine.cpp', 'w') as outfile:
    outfile.write(generate_subtroutine())
#print(generate_subtroutine())

In [17]:
print(generate_subtroutine())

int ComputeSYMGS_ref( const SparseMatrix & A, const Vector & r, Vector & x) {

  assert(x.localLength==A.localNumberOfColumns); // Make sure x contain space for halo values

#ifndef HPCG_NO_MPI
  ExchangeHalo(A,x);
#endif

  local_int_t ix = 0;
  local_int_t iy = 0;
  local_int_t iz = 0;
  local_int_t nex = 0;
  local_int_t ney = 0;
  local_int_t nez = 0;
  local_int_t nx = A.geom->nx;
  local_int_t ny = A.geom->ny;
  local_int_t nz = A.geom->nz;
  local_int_t nlocal = nx*ny*nz;
  int npx          = A.geom->npx;
  int npy          = A.geom->npy;
  int npz          = A.geom->npz;
  int ipx          = A.geom->ipx;
  int ipy          = A.geom->ipy;
  int ipz          = A.geom->ipz;

//  const local_int_t nrow = A.localNumberOfRows;
//  double ** matrixDiagonal = A.matrixDiagonal;  // An array of pointers to the diagonal entries A.matrixValues
  const double * const rv = r.values;
  double * const xv = x.values;

// forward sweep
for( iz=0; iz< nz; iz++){
    for( iy=0; iy< ny; iy++){
    